In [18]:

fopen = open("20221001.as-rel2.txt", encoding="utf-8")

relationships = {}

for line in fopen:
    if line[0] != "#":
        line_split = line.split("|")
        as1 = line_split[0].strip("\n")
        as2 = line_split[1].strip("\n")
        reltype = int(line_split[2].strip("\n"))
        if as1 not in relationships:
            # [customer array, provider array]
            relationships[as1] = [set(), set()]
        if as2 not in relationships:
            # [customer array, provider array]
            relationships[as2] = [set(), set()]
        # If p2c
        if reltype == -1:
            # as1 has customer
            relationships[as1][0].add(as2)
            # as2 has a provider
            relationships[as2][1].add(as1)

fopen.close()

# Find all AS's with no children
customers = {}
as_list = set()
for as_name in relationships:
    as_data = relationships[as_name]
    if len(as_data[0]) == 0:
        customers[as_name] = as_data[0]
        as_list.add(as_name)

for as_name in as_list:
    parents = relationships[as_name][1]
    for parent in parents:
        if parent in customers:
            customers[parent].add(as_name)
        else:
            customers[parent] = {as_name}
        as_list.add(as_name)
        for c in customers[as_name]:
            customers[parent].add(c)
            as_list.add(as_name)
sorted_customers = sorted(customers.items(), key=lambda key: len(key[1]), reverse=True)
total_as = 0
total_prefix = 0
total_ip = 0
as_stats = {}
for as_num in sorted_customers:
    as_stats[as_num[0]] = {}
    as_stats[as_num[0]]["as_number"] = len(as_num[1])
    total_as += len(as_num[1])

fopen = open("routeviews-rv2-20221001-1200.pfx2as", encoding="utf-8")

ipspace = {}

for line in fopen:
    if line[0] != "#":
        line_split = line.split()
        ip = line_split[0]
        prefix = int(line_split[1])
        as1 = line_split[2].strip("\n")
        if as1 not in ipspace:
            ipspace[as1] = [0, 0]
        ipspace[as1][0] += prefix
        ip_calc = 2 ** (32 - prefix)
        ipspace[as1][1] += ip_calc
        total_prefix += prefix
        total_ip += ip_calc
for as_num in sorted_customers:
    if as_num[0] in ipspace:
        as_stats[as_num[0]]["prefix"] = ipspace[as_num[0]][0]
        as_stats[as_num[0]]["ip"] = ipspace[as_num[0]][1]
    else:
        as_stats[as_num[0]]["prefix"] = 0
        as_stats[as_num[0]]["ip"] = 0

    